## Configuration

In [12]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *
from datetime import date

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Old Playlists
playlistsOld = [
    'Summer 2018',
    'Summer 2019',
    'Endless Summer 2020',
    'Endless Summer 2021',
    'Endless Summer 2022',
    'Endless Summer 2023'
]

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')
artistsHistoricalPath = os.path.join('data', 'artistsHistorical.csv')

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in Known Artists

In [3]:
# Read in known artists from csv
if os.path.exists(artistsKnownPath):
    artistsKnown = pd.read_csv(artistsKnownPath)
else:
    artistsKnown = pd.DataFrame(columns=['artistID', 'following'])

## Read in old playlists

In [4]:
# Empty df
artistsOld = pd.DataFrame()
# Read in all old summer playlists
for playlistName in playlistsOld:
    # Get playlistID
    id_pl = getPlaylistID(credentials, playlistName)
    # Get playlistInfo
    tracks_pl = fetchTracksFromPlaylist(credentials, id_pl)
    # Append to df
    artistsOld = pd.concat([artistsOld, tracks_pl], axis=0)
# Deduplicate
artistsOld = artistsOld.drop_duplicates('artistID', ignore_index=True)[['artistID']]

## Merge and deduplicate known artists and any from new playlists

In [5]:
artistsKnown = pd.concat([artistsKnown, artistsOld], axis=0).drop_duplicates('artistID', ignore_index=True)
# Mark Liked and Unliked artists
artistsKnown.loc[artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = True
artistsKnown.loc[~artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = False
artistsKnown

## Get artist info and add it to historical data

In [6]:
# Get artist info based on list of IDs
artistInfo = fetchArtistsInfo(credentials, artistsKnown['artistID'].to_list())
# Set the date of the data capture
artistInfo['date'] = date.today()
# Record whether artist was liked or not
artistInfo = artistInfo.merge(artistsKnown, how='left', on='artistID')

# Read in historical
artistsHistorical = pd.read_csv(artistsHistoricalPath)

# Merge in new data
artistsHistorical = pd.concat([artistsHistorical, artistInfo], axis=0)
artistsHistorical

## Load

In [13]:
# Write artistsKnown and artistsHistorical csvs
artistInfo.to_csv(artistsKnownPath, index=False)
artistsHistorical.to_csv(artistsHistoricalPath, index=False)